Paso 1: Importación de datos

In [1]:
# Librerías necesarias (Se añadirán más durante el proyecto):
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('datasets/listado.csv', sep=',', encoding='latin-1')

# Para hacer display de todas las columnas que hay en el dataset
pd.set_option('display.max_columns', None)

df


,Codigo,Aseos,Baños,Balcon,Certificado Energetico,Cocina,Comunidad Incluida,Estado Ficha,Estado Propiedad,Garaje,Habitaciones Total,IBI,Mt Construidos,Mt Parcela,Mt Utiles,Muebles,Parking,Salón,IVA,Precio a Consultar,Precio Alquiler,Precio Anterior,Precio Inmobiliaria,Precio IVA,Precio Propietario,Precio Traspaso,CP,Agua,Aire Acondicionado,Aire Acond. Central,Alarma,Ascensor,Autobuses,Calefacción,Calefacción central,Céntrico,Centros comerciales,Centros médicos,Cerca de Universidad,Colegios,Jardín,Patio,Terraza
0,16186583,1,1,1,Trámites,Equipada,0,Fin de Encargo,Buen estado,sin garaje,6,0.0,154.0,132,154.0,1,Sin Parking,2,21,0,0,0,93300,588,89913,0,29420,1,0,0,0,0,1,1,0,1,0,1,0,1,1,1,1
1,16186768,0,2,0,Trámites,Equipada,0,Fin de Encargo,Entrar a vivir,Garaje Incluido,3,0.0,118.0,0,90.0,1,Parking incluido,1,21,0,0,217600,204000,1285,196595,0,29649,1,0,0,0,1,1,1,0,1,1,1,0,1,0,0,1
2,16187000,1,2,0,Trámites,NaN,0,Libre,Entrar a vivir,Garaje Incluido,3,0.0,200.0,300,200.0,1,Parking incluido,1,21,0,0,0,170000,1071,163829,0,29410,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,16187920,0,0,0,Exento,NaN,0,Libre,Ninguno,sin garaje,0,0.0,0.0,245,0.0,0,Sin Parking,0,21,0,0,0,95000,599,91552,0,29420,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0
4,16188188,1,1,0,Trámites,Equipada,0,Libre,Entrar a vivir,sin garaje,4,0.0,200.0,100,200.0,1,Sin Parking,2,21,0,0,0,83000,523,79987,0,29410,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,19382161,1,1,0,Pendiente,Equipada,0,Libre,Buen estado,sin garaje,4,0.0,172.0,80,172.0,0,Sin Parking,2,21,0,0,0,77000,1260,69740,0,29109,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1
110,19448234,0,1,0,Pendiente,Solo Muebles,0,Libre,Buen estado,sin garaje,2,0.0,50.0,6000,50.0,0,Sin Parking,1,21,0,0,0,127000,1260,119740,0,29109,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
111,19448337,0,1,1,Pendiente,Solo Muebles,0,Libre,Buen estado,sin garaje,3,0.0,122.0,74,122.0,1,Sin Parking,1,21,0,0,0,87000,1260,79740,0,29410,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1
112,19448381,1,2,1,Pendiente,Solo Muebles,0,Libre,Buen estado,sin garaje,5,0.0,146.0,65,146.0,0,Sin Parking,2,21,0,0,0,97000,1260,89740,0,29410,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1


Paso 2: Análisis de importancia de propiedades y datos

In [3]:
# Cambio de los nombres de las columnas a minúsculas y sustitución de espacios por guiones bajos en los valores
replacer = lambda str : str.lower().str.replace(" ","_")
df.columns = replacer(df.columns.str)
for col in list(df.dtypes[df.dtypes == "object"].index):
    df[col] = replacer(df[col].str)
df.head().T

,0,1,2,3,4
codigo,16186583,16186768,16187000,16187920,16188188
aseos,1,0,1,0,1
baños,1,2,2,0,1
balcon,1,0,0,0,0
certificado_energetico,trámites,trámites,trámites,exento,trámites
cocina,equipada,equipada,NaN,NaN,equipada
comunidad_incluida,0,0,0,0,0
estado_ficha,fin_de_encargo,fin_de_encargo,libre,libre,libre
estado_propiedad,buen_estado,entrar_a_vivir,entrar_a_vivir,ninguno,entrar_a_vivir
garaje,sin_garaje,garaje_incluido,garaje_incluido,sin_garaje,sin_garaje


In [4]:
# Rellenamos valores nulos

df = df.fillna("no_info")

In [5]:
# Eliminamos atributos no relevantes y/o repetidos
df = df.drop(['codigo', 'ibi', 'certificado_energetico'], axis=1)

In [6]:
# Cambio de tipo de columnas a categórico puesto que hay atributos con valores 0-1

df['agua']=pd.Categorical(df['agua'])

df['aire_acond._central']=pd.Categorical(df['aire_acond._central'])

df['aire_acondicionado']=pd.Categorical(df['aire_acondicionado'])

df['ascensor']=pd.Categorical(df['ascensor'])

df['autobuses']=pd.Categorical(df['autobuses'])

df['alarma']=pd.Categorical(df['alarma'])

df['balcon']=pd.Categorical(df['balcon'])

df['calefacción']=pd.Categorical(df['calefacción'])

df['calefacción_central']=pd.Categorical(df['calefacción_central'])

df['centros_médicos']=pd.Categorical(df['centros_médicos'])

df['cerca_de_universidad']=pd.Categorical(df['cerca_de_universidad'])

df['colegios']=pd.Categorical(df['colegios'])

df['comunidad_incluida']=pd.Categorical(df['comunidad_incluida'])

df['jardín']=pd.Categorical(df['jardín'])

df['patio']=pd.Categorical(df['patio'])

df['terraza']=pd.Categorical(df['terraza'])

df['céntrico']=pd.Categorical(df['céntrico'])

df['centros_comerciales']=pd.Categorical(df['centros_comerciales'])

df['muebles']=pd.Categorical(df['muebles'])

In [7]:
# Identificamos el atributo target (Estado Ficha) y reemplazamos sus valores para hacerlo un atributo target dicotómico

propiedades_estado_ficha = ["vendida","vendida_por_otros","pendiente_de_firma","señalizada","alquilada_por_otros","contrato_arras","fin_de_encargo"]

df['estado_ficha'] = df['estado_ficha'].apply(lambda x: 1 if x in propiedades_estado_ficha else 0)

df['estado_ficha']=pd.Categorical(df['estado_ficha'])

In [8]:
# Reemplazamos otros valores para hacerlos atributos dicotómicos

# Cocina - Equipada, Solo muebles, = 1, no_info, Vacia = 0

propiedades_cocina = ["equipada", "solo_muebles"]

df['cocina'] = df['cocina'].apply(lambda x: 1 if x in propiedades_cocina else 0)

df['cocina']=pd.Categorical(df['cocina'])

# Estado propiedad - Buen estado, Entrar a Vivir, Reformado, Nuevo, Buen estado = 1 , lo demás 0

propiedades_estado = ["buen_estado", "entrar_a_vivir", "reformado", "nuevo", "buen_estado"]

df['estado_propiedad'] = df['estado_propiedad'].apply(lambda x: 1 if x in propiedades_estado else 0)

df['estado_propiedad']=pd.Categorical(df['estado_propiedad'])

# Garaje - Garaje Incluido = 1, sin garaje = 0

propiedades_garaje = ["garaje_incluido"]

df['garaje'] = df['garaje'].apply(lambda x: 1 if x in propiedades_garaje else 0)

df['garaje']=pd.Categorical(df['garaje'])

# Parking - Parking Incluido = 1, sin parking = 0

propiedades_parking = ["parking_incluido"]

df['parking'] = df['parking'].apply(lambda x: 1 if x in propiedades_parking else 0)

df['parking']=pd.Categorical(df['parking'])

In [9]:
# Convert single column to int dtype.
df['cp'] = df['cp'].astype('int')

df['mt_construidos'] = df['mt_construidos'].astype('int')

df['mt_utiles'] = df['mt_utiles'].astype('int')

In [10]:
df

,aseos,baños,balcon,cocina,comunidad_incluida,estado_ficha,estado_propiedad,garaje,habitaciones_total,mt_construidos,mt_parcela,mt_utiles,muebles,parking,salón,iva,precio_a_consultar,precio_alquiler,precio_anterior,precio_inmobiliaria,precio_iva,precio_propietario,precio_traspaso,cp,agua,aire_acondicionado,aire_acond._central,alarma,ascensor,autobuses,calefacción,calefacción_central,céntrico,centros_comerciales,centros_médicos,cerca_de_universidad,colegios,jardín,patio,terraza
0,1,1,1,1,0,1,1,0,6,154,132,154,1,0,2,21,0,0,0,93300,588,89913,0,29420,1,0,0,0,0,1,1,0,1,0,1,0,1,1,1,1
1,0,2,0,1,0,1,1,1,3,118,0,90,1,1,1,21,0,0,217600,204000,1285,196595,0,29649,1,0,0,0,1,1,1,0,1,1,1,0,1,0,0,1
2,1,2,0,0,0,0,1,1,3,200,300,200,1,1,1,21,0,0,0,170000,1071,163829,0,29410,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,0,0,0,0,0,0,0,0,0,0,245,0,0,0,0,21,0,0,0,95000,599,91552,0,29420,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0
4,1,1,0,1,0,0,1,0,4,200,100,200,1,0,2,21,0,0,0,83000,523,79987,0,29410,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1,1,0,1,0,0,1,0,4,172,80,172,0,0,2,21,0,0,0,77000,1260,69740,0,29109,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1
110,0,1,0,1,0,0,1,0,2,50,6000,50,0,0,1,21,0,0,0,127000,1260,119740,0,29109,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1
111,0,1,1,1,0,0,1,0,3,122,74,122,1,0,1,21,0,0,0,87000,1260,79740,0,29410,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1
112,1,2,1,1,0,0,1,0,5,146,65,146,0,0,2,21,0,0,0,97000,1260,89740,0,29410,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,1


In [11]:
x = df.drop(['estado_ficha'], axis = 1)

y = df['estado_ficha']

Paso 3: Ingeniería de propiedades.

In [12]:
# Sustitución de los valores de la variable objetivo por valores numéricos

df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

df_test, df_val = train_test_split(df_train_full, test_size=0.33, random_state=42)

y_train = df_test.estado_ficha.values
y_testing = df_val.estado_ficha.values

del df_test['estado_ficha']
del df_val['estado_ficha']

df_test.head().T

,29,44,17,97,42
aseos,0,0,0,2,2
baños,1,0,1,1,1
balcon,0,0,0,1,0
cocina,1,0,0,0,1
comunidad_incluida,0,0,0,0,0
estado_propiedad,1,0,1,1,1
garaje,1,0,0,0,1
habitaciones_total,1,0,3,5,5
mt_construidos,80,0,74,151,445
mt_parcela,5400,201,0,118,445


In [13]:
# Cálculo de la información mutua entre la variable objetivo y el resto de variables categóricas
calculate_mi = lambda col: mutual_info_score(col, df_train_full.estado_ficha)

categorical = ['aseos','baños','comunidad_incluida','estado_propiedad' ,'garaje','muebles', 'parking','balcon','cocina',
            'agua','aire_acondicionado','aire_acond._central','alarma','ascensor','autobuses','calefacción','calefacción_central','céntrico',
            'centros_comerciales','centros_médicos','cerca_de_universidad','colegios','jardín','patio','terraza']

numerical = ['habitaciones_total','mt_construidos','mt_parcela','mt_utiles','salón','iva','precio_a_consultar','precio_alquiler',
            'precio_anterior','precio_inmobiliaria','precio_iva','precio_propietario','precio_traspaso','cp', ]

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name="MI")
df_mi

,MI
centros_comerciales,0.062848
ascensor,0.022570
aseos,0.021693
balcon,0.020097
baños,0.014755
centros_médicos,0.014625
cerca_de_universidad,0.014360
alarma,0.014360
colegios,0.012994
céntrico,0.011101


In [14]:
# Conversión del dataset a una lista de diccionarios

train_dict = df_test[categorical + numerical].to_dict(orient="records")
train_dict[0]

{'aseos': 0,
 'baños': 1,
 'comunidad_incluida': 0,
 'estado_propiedad': 1,
 'garaje': 1,
 'muebles': 0,
 'parking': 1,
 'balcon': 0,
 'cocina': 1,
 'agua': 1,
 'aire_acondicionado': 0,
 'aire_acond._central': 0,
 'alarma': 0,
 'ascensor': 0,
 'autobuses': 0,
 'calefacción': 1,
 'calefacción_central': 0,
 'céntrico': 0,
 'centros_comerciales': 0,
 'centros_médicos': 0,
 'cerca_de_universidad': 0,
 'colegios': 0,
 'jardín': 1,
 'patio': 1,
 'terraza': 1,
 'habitaciones_total': 1,
 'mt_construidos': 80,
 'mt_parcela': 5400,
 'mt_utiles': 75,
 'salón': 1,
 'iva': 21,
 'precio_a_consultar': 0,
 'precio_alquiler': 0,
 'precio_anterior': 0,
 'precio_inmobiliaria': 106700,
 'precio_iva': 672,
 'precio_propietario': 102827,
 'precio_traspaso': 0,
 'cp': 29410}

In [15]:
# Transformación de la lista de diccionarios a una matriz

dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [16]:
X_train = dv.transform(train_dict)
X_train[0]

array([1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00, 1.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00,
       0.00000e+00, 0.00000e+00, 2.94100e+04, 0.00000e+00, 1.00000e+00,
       1.00000e+00, 1.00000e+00, 2.10000e+01, 1.00000e+00, 8.00000e+01,
       5.40000e+03, 7.50000e+01, 0.00000e+00, 1.00000e+00, 1.00000e+00,
       0.00000e+00, 0.00000e+00, 0.00000e+00, 1.06700e+05, 6.72000e+02,
       1.02827e+05, 0.00000e+00, 1.00000e+00, 1.00000e+00])

In [17]:
# Array con nombre de los atributos
dv.get_feature_names_out()

array(['agua', 'aire_acond._central', 'aire_acondicionado', 'alarma',
       'ascensor', 'aseos', 'autobuses', 'balcon', 'baños', 'calefacción',
       'calefacción_central', 'centros_comerciales', 'centros_médicos',
       'cerca_de_universidad', 'cocina', 'colegios', 'comunidad_incluida',
       'cp', 'céntrico', 'estado_propiedad', 'garaje',
       'habitaciones_total', 'iva', 'jardín', 'mt_construidos',
       'mt_parcela', 'mt_utiles', 'muebles', 'parking', 'patio',
       'precio_a_consultar', 'precio_alquiler', 'precio_anterior',
       'precio_inmobiliaria', 'precio_iva', 'precio_propietario',
       'precio_traspaso', 'salón', 'terraza'], dtype=object)

Paso 4: Entrenamiento del modelo

In [18]:
# Utilizando SVC para nuestro modelo

from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = svm.SVC(gamma='scale', probability=True, kernel="sigmoid")

model.fit(X_train, y_train)

predicted = model.predict(df_test)

c:\Users\34688\AppData\Local\pypoetry\Cache\virtualenvs\svm-streamlist-test-x4EsxH3F-py3.11\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


In [19]:
# Realizando predicciones del modelo sobre el conjunto de datos de validación

val_dict = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)
y_pred = model.predict_proba(X_val)

y_pred

array([[0.81435849, 0.18564151],
       [0.75232342, 0.24767658],
       [0.7336448 , 0.2663552 ],
       [0.79482441, 0.20517559],
       [0.78479646, 0.21520354],
       [0.74218958, 0.25781042],
       [0.76038271, 0.23961729],
       [0.72425119, 0.27574881],
       [0.75287099, 0.24712901],
       [0.74796974, 0.25203026],
       [0.74020319, 0.25979681],
       [0.80668089, 0.19331911],
       [0.74012934, 0.25987066],
       [0.83644371, 0.16355629],
       [0.73928715, 0.26071285],
       [0.73996878, 0.26003122],
       [0.74698512, 0.25301488],
       [0.73133427, 0.26866573],
       [0.7483349 , 0.2516651 ],
       [0.74400963, 0.25599037],
       [0.72559561, 0.27440439],
       [0.76040504, 0.23959496],
       [0.74231071, 0.25768929],
       [0.73465083, 0.26534917],
       [0.73415846, 0.26584154],
       [0.74416712, 0.25583288],
       [0.76078423, 0.23921577],
       [0.81514035, 0.18485965],
       [0.74860746, 0.25139254],
       [0.80890901, 0.19109099],
       [0.

In [20]:
# Simplificando las probabilidades obtenidas de las predicciones

y_pred = model.predict_proba(X_val)[:,1]

Se_Vende = y_pred >= 0.5

Se_Vende

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [21]:
# Predicción al 64% en la mayoría de kernel, con poly directamente no realiza el modelo, tarda muchas horas.
round((y_testing == Se_Vende).mean(),3)

0.645

Paso 5: Serialización del modelo con Pickle

In [22]:
# Importamos pickle y creamos el .pck "supervivientes" dentro de la carpeta "models"
import pickle
with open("models/se_vende.pck", "wb") as f:
    pickle.dump((dv, model), f)

In [23]:
with open("models/se_vende.pck", "rb") as f:
    dv, modeling = pickle.load(f)
    X_val = dv.transform(val_dict)
    y_pred = modeling.predict_proba(X_val)

y_pred

array([[0.81435849, 0.18564151],
       [0.75232342, 0.24767658],
       [0.7336448 , 0.2663552 ],
       [0.79482441, 0.20517559],
       [0.78479646, 0.21520354],
       [0.74218958, 0.25781042],
       [0.76038271, 0.23961729],
       [0.72425119, 0.27574881],
       [0.75287099, 0.24712901],
       [0.74796974, 0.25203026],
       [0.74020319, 0.25979681],
       [0.80668089, 0.19331911],
       [0.74012934, 0.25987066],
       [0.83644371, 0.16355629],
       [0.73928715, 0.26071285],
       [0.73996878, 0.26003122],
       [0.74698512, 0.25301488],
       [0.73133427, 0.26866573],
       [0.7483349 , 0.2516651 ],
       [0.74400963, 0.25599037],
       [0.72559561, 0.27440439],
       [0.76040504, 0.23959496],
       [0.74231071, 0.25768929],
       [0.73465083, 0.26534917],
       [0.73415846, 0.26584154],
       [0.74416712, 0.25583288],
       [0.76078423, 0.23921577],
       [0.81514035, 0.18485965],
       [0.74860746, 0.25139254],
       [0.80890901, 0.19109099],
       [0.